# Instruction Finetuning


### 데이터셋 구축

1. 목적 정의: 먼저, 세부 튜닝을 통해 달성하고자 하는 목표를 명확히 합니다.
1. 데이터 수집: 목표에 맞는 데이터를 수집합니다. 이 데이터는 공개 데이터셋일 수도 있고, 사용자가 직접 수집한 데이터일 수도 있습니다.

1. 데이터 가공: 수집한 데이터를 모델 훈련에 적합하게 가공합니다. 이 과정에서는 데이터를 정제하고, 필요한 형식으로 변환하는 작업이 포함됩니다.

### 공개 데이터셋 다운로드

In [21]:
!pip install -U datasets==2.17.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.5.0
    Uninstalling fsspec-2024.5.0:
      Successfully uninstalled fsspec-2024.5.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.20.0
    Uninstalling datasets-2.20.0:
      Successfully uninstalled datasets-2.20.0


In [1]:
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("royboy0416/ko-alpaca")

# 데이터셋의 구조 확인
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 49620
    })
})


In [2]:
dataset['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}

# Gemma 데이터셋 포맷팅

```<start_of_turn>user```<br>
```What is Cramer's Rule?<end_of_turn>```<br>
```<start_of_turn>model```<br>
```Cramer's Rule is ...<end_of_turn>```

In [3]:
# 'prompt' 필드 생성 함수
def format_instruction(example):

    # 추가 컨텍스트(input 필드)가 있는 경우
    if example['input'] and len(example['input']) > 0:
        text = f"""<start_of_turn>user\n{example["instruction"]}\n{example["input"]}<end_of_turn>\n<start_of_turn>model\n{example["output"]}<end_of_turn>"""
    # input 필드가 없는 경우
    else:
        text = f"""<start_of_turn>user\n{example["instruction"]}<end_of_turn>\n<start_of_turn>model\n{example["output"]}<end_of_turn>"""

    return {'prompt': text}

# 데이터셋의 prompt 필드를 업데이트
dataset = dataset.map(format_instruction)

In [4]:
dataset['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'prompt': '<start_of_turn>user\n건강을 유지하기 위한 세 가지 팁을 알려주세요.<end_of_turn>\n<start_of_turn>model\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.<end_of_turn>'}

In [5]:
dataset['train'][5]

{'instruction': '홀수 중 하나를 밝히세요.',
 'input': '트위터, 인스타그램, 텔레그램',
 'output': '텔레그램입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n홀수 중 하나를 밝히세요.\n\n### Input:\n트위터, 인스타그램, 텔레그램\n\n### Response:\n텔레그램입니다.',
 'prompt': '<start_of_turn>user\n홀수 중 하나를 밝히세요.\n트위터, 인스타그램, 텔레그램<end_of_turn>\n<start_of_turn>model\n텔레그램입니다.<end_of_turn>'}

### 모델 로드 및 튜닝:

1. 모델 학습: gemma-2b 모델을 로드하고, 준비된 데이터셋을 사용하여 모델을 세부 튜닝합니다. 이 과정에서는 학습률, 에폭 수 등의 파라미터를 조정할 수 있습니다.
1. 평가 및 반복: 튜닝된 모델을 평가하고 결과를 확인합니다. 필요에 따라 여러 번 반복하여 모델의 성능을 최적화할 수 있습니다.

In [22]:
!pip install -qU transformers==4.38.0 accelerate==0.27.1 bitsandbytes==0.42.0 peft==0.8.2 trl==0.7.10


In [23]:
!pip install accelerate
!pip install bitsandbytes --upgrade

In [6]:
!nvidia-smi

Sat Jun 15 21:51:45 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0 Off |                  N/A |
| 50%   35C    P8             11W /  210W |    1564MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login

In [25]:
# 자기 자신의 허깅페이스 토큰 필요
notebook_login()

In [29]:
model_id = "google/gemma-2b"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)


# model = AutoModelForCausalLM.from_pretrained(model_id,
#                                              quantization_config=bnb_config,
#                                              device_map={"":0})

# tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

In [30]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/Users/leeshinhaeng/opt/anaconda3/envs/llm24/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [11]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 39696
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 9924
    })
})

In [12]:
train_data = dataset["train"]
test_data = dataset["test"]

In [13]:
print(train_data[0])

{'instruction': '이것은 개방형 생성 과제입니다. GPT 모델은 명령에 적합한 출력을 생성해야 합니다.', 'input': '면접을 준비하는 방법', 'output': '면접을 준비하기 위해 관련된 질문 목록을 작성하고, 효과적인 답변 전략을 개발하여 자신감 있게 나설 수 있도록 해보세요.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n이것은 개방형 생성 과제입니다. GPT 모델은 명령에 적합한 출력을 생성해야 합니다.\n\n### Input:\n면접을 준비하는 방법\n\n### Response:\n면접을 준비하기 위해 관련된 질문 목록을 작성하고, 효과적인 답변 전략을 개발하여 자신감 있게 나설 수 있도록 해보세요.', 'prompt': '<start_of_turn>user\n이것은 개방형 생성 과제입니다. GPT 모델은 명령에 적합한 출력을 생성해야 합니다.\n면접을 준비하는 방법<end_of_turn>\n<start_of_turn>model\n면접을 준비하기 위해 관련된 질문 목록을 작성하고, 효과적인 답변 전략을 개발하여 자신감 있게 나설 수 있도록 해보세요.<end_of_turn>', 'input_ids': [2, 106, 1645, 108, 235832, 139988, 49532, 238037, 238867, 116518, 65084, 236939, 47555, 235265, 162174, 162570, 236648, 95165, 240446, 236179, 99797, 237961, 236511, 182260, 236392, 116518, 149735, 179694, 235265, 108, 237722, 240449, 236392, 166422, 237584, 40284, 130059, 107, 108, 106, 2516, 108, 23

In [14]:
def get_completion(query: str, model, tokenizer):

  prompt_template = """<start_of_turn>user
  {query}
  <end_of_turn>
  <start_of_turn>model
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, max_new_tokens=256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

# Fine tuning 이전
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.", model=model, tokenizer=tokenizer)
print(result)


c:\Users\mkh08\OneDrive\문서\GitHub\Ollama_TestCode\venv\Lib\site-packages\transformers\models\gemma\modeling_gemma.py:555: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


user
  건강을 유지하기 위한 세 가지 팁을 알려주세요.
  
  model
  
  abbaye
  이것은 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 


In [15]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

c:\Users\mkh08\OneDrive\문서\GitHub\Ollama_TestCode\venv\Lib\site-packages\trl\trainer\sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/39696 [00:00<?, ? examples/s]

Map:   0%|          | 0/9924 [00:00<?, ? examples/s]

c:\Users\mkh08\OneDrive\문서\GitHub\Ollama_TestCode\venv\Lib\site-packages\trl\trainer\sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

{'loss': 3.7332, 'grad_norm': 0.8657228350639343, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 3.0952, 'grad_norm': 1.0215117931365967, 'learning_rate': 0.00017777777777777779, 'epoch': 0.0}
{'loss': 2.6608, 'grad_norm': 1.099008560180664, 'learning_rate': 0.00015555555555555556, 'epoch': 0.0}
{'loss': 2.3759, 'grad_norm': 0.8393830060958862, 'learning_rate': 0.00013333333333333334, 'epoch': 0.0}
{'loss': 2.2601, 'grad_norm': 1.1383107900619507, 'learning_rate': 0.00011111111111111112, 'epoch': 0.01}
{'loss': 2.3429, 'grad_norm': 0.7046269178390503, 'learning_rate': 8.888888888888889e-05, 'epoch': 0.01}
{'loss': 2.3607, 'grad_norm': 2.020468235015869, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.01}
{'loss': 2.1884, 'grad_norm': 0.699824869632721, 'learning_rate': 4.4444444444444447e-05, 'epoch': 0.01}
{'loss': 2.1641, 'grad_norm': 0.8179895281791687, 'learning_rate': 2.2222222222222223e-05, 'epoch': 0.01}
{'loss': 2.2895, 'grad_norm': 0.5826743841171265, 'learning_rate': 0.0,

TrainOutput(global_step=100, training_loss=2.547081050872803, metrics={'train_runtime': 56.9655, 'train_samples_per_second': 7.022, 'train_steps_per_second': 1.755, 'train_loss': 2.547081050872803, 'epoch': 0.01})

In [16]:
# Fine tuning 이후
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  건강을 유지하기 위한 세 가지 팁을 알려주세요.
  
  model
   1. 규칙적인 운동을 하세요.
  2. 건강한 식사를 즐기세요.
  3. 규칙적인 수면을 취하세요.
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur: 3
   Vainqueur:


In [17]:
# Fine tuning 이후
result = get_completion(query="불면증을 해결하는 방법을 세 가지 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  불면증을 해결하는 방법을 세 가지 알려주세요.
  
  model
   1. 불면증을 해결하는 방법을 세 가지 알려주세요.
  2. 불면증을 해결하는 방법을 세 가지 알려주세요.
  3. 불면증을 해결하는 방법을 세 가지 알려주세요.
  :+::+: 델타:+:
  :+:+:+:+: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+::::: 델타:+:
  :+:::::


# 모델 저장

In [18]:
new_model = "gemma-2b-it-koalpaca-finetuned"
trainer.model.save_pretrained(new_model)